<a href="https://colab.research.google.com/github/mohammadRahimi1993/Exercise/blob/main/LinkPrediction_TransE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LinkPrediction_TransE**

In [ ]:
!pip install numpy
!pip install torch
!pip install scikit-learn

**Import Library**

In [15]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

**TransE**

In [16]:
# تعریف مدل TransE
class TransE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(TransE, self).__init__()
        self.entity_embeddings = nn.Embedding(num_entities, embedding_dim)
        self.relation_embeddings = nn.Embedding(num_relations, embedding_dim)
        self.embedding_dim = embedding_dim

        # مقداردهی اولیه تصادفی برای تعبیه‌ها
        nn.init.uniform_(self.entity_embeddings.weight, -6/np.sqrt(embedding_dim), 6/np.sqrt(embedding_dim))
        nn.init.uniform_(self.relation_embeddings.weight, -6/np.sqrt(embedding_dim), 6/np.sqrt(embedding_dim))

    def forward(self, head, relation, tail, mode='train'):
        head_emb = self.entity_embeddings(head)
        relation_emb = self.relation_embeddings(relation)
        tail_emb = self.entity_embeddings(tail)

        if mode == 'train':
            return head_emb + relation_emb - tail_emb
        else:
            return torch.norm(head_emb + relation_emb - tail_emb, p=2, dim=1)

**loss_function + get_data**

In [17]:
# تعریف تابع تلف زیان
def loss_function(pos_score, neg_score, margin=1.0):
    return torch.sum(torch.relu(pos_score - neg_score + margin))

# آماده‌سازی داده‌ها
def get_data():
    # تعداد موجودیت‌ها و روابط
    num_entities = 6
    num_relations = 2

    # داده‌های آموزشی (سه‌تایی‌ها)
    triples = [
        (0, 0, 1),
        (0, 1, 2),
        (1, 0, 3),
        (2, 1, 3),
        (4, 0, 5)
    ]

    return triples, num_entities, num_relations

**Train_model**

In [18]:
# تابع آموزش مدل
def train_model(model, triples, num_epochs=100, learning_rate=0.01):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        epoch_loss = 0
        for head, relation, tail in triples:
            head = torch.LongTensor([head])
            relation = torch.LongTensor([relation])
            tail = torch.LongTensor([tail])

            # پیش‌بینی برای سه‌تایی‌های مثبت و منفی
            pos_score = model(head, relation, tail)
            neg_tail = torch.randint(0, model.entity_embeddings.num_embeddings, tail.size())
            neg_score = model(head, relation, neg_tail)

            # محاسبه زیان و به‌روزرسانی وزن‌ها
            loss = loss_function(pos_score, neg_score)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

**Evalute_Model**

In [19]:
# تابع ارزیابی مدل
def evaluate_model(model, triples):
    model.eval()
    with torch.no_grad():
        y_true = []
        y_pred = []

        for head, relation, tail in triples:
            head = torch.LongTensor([head])
            relation = torch.LongTensor([relation])
            tail = torch.LongTensor([tail])

            pos_score = model(head, relation, tail, mode='eval').item()
            neg_tail = torch.randint(0, model.entity_embeddings.num_embeddings, tail.size())
            neg_score = model(head, relation, neg_tail, mode='eval').item()

            y_true.append(1)
            y_pred.append(1 if pos_score < neg_score else 0)

        accuracy = accuracy_score(y_true, y_pred)
        print(f'Accuracy: {accuracy:.4f}')

**Run script**

In [20]:
# اجرای اسکریپت
triples, num_entities, num_relations = get_data()
model = TransE(num_entities, num_relations, embedding_dim=50)
train_model(model, triples)
evaluate_model(model, triples)

Epoch 10/100, Loss: 232.1065
Epoch 20/100, Loss: 225.6455
Epoch 30/100, Loss: 136.0719
Epoch 40/100, Loss: 234.0056
Epoch 50/100, Loss: 237.5323
Epoch 60/100, Loss: 133.7335
Epoch 70/100, Loss: 209.4912
Epoch 80/100, Loss: 75.7606
Epoch 90/100, Loss: 163.3373
Epoch 100/100, Loss: 109.8516
Accuracy: 0.2000
